In [15]:
import pandas as pd
import numpy as np

In [16]:
defaults = pd.read_csv('../../rawdata/weather/weather.defaults')
defaults = defaults.columns.values

def removeErrorValues(row):
    
    if(row > 30 and str(row) in defaults):
        return 0
    else:
        return row

In [17]:
cols=[0,11, 28]
weatherDFrame_header = pd.read_csv('../../rawdata/weather/weather.header', usecols = cols)
weatherDFrame = pd.read_csv('../../rawdata/weather/weather', usecols = cols, names=weatherDFrame_header.columns.values)
weatherDFrame['datetime'] = pd.to_datetime(weatherDFrame['time'],unit='s')
weatherDFrame['datetime'].dt.tz_localize('utc').dt.tz_convert('US/Eastern')

weatherDFrame.sort_values(by=['datetime'], inplace=True)
weatherDFrame.set_index(['datetime'], inplace = True)
weatherDFrame.drop(['time'], axis=1, inplace=True)

In [18]:
### cleaning default values
weatherDFrame['Spd[Wind]'] = weatherDFrame.apply(lambda row: removeErrorValues(row['Spd[Wind]']), axis=1)
weatherDFrame['Amt[PrecipHourly1]'] = weatherDFrame.apply(lambda row: removeErrorValues(row['Amt[PrecipHourly1]']), axis=1)

In [21]:
weatherDFrame.tail()

,Spd[Wind],Amt[PrecipHourly1]
datetime,,
2018-04-01 21:51:00,7.7,0.0
2018-04-01 22:00:00,7.7,0.0
2018-04-01 22:51:00,7.2,0.0
2018-04-01 23:51:00,6.7,0.0
2018-04-02 00:51:00,7.2,0.0


In [20]:
precipitation = weatherDFrame[['Amt[PrecipHourly1]']]
wind = weatherDFrame[['Spd[Wind]']]

precipitation.to_pickle('../../data/weather/precipitation.pkl')
wind.to_pickle('../../data/weather/wind.pkl')

In [24]:
precipitation.head()

,Amt[PrecipHourly1]
datetime,
2010-01-01 01:00:00,0.0
2010-01-01 01:15:00,0.0
2010-01-01 01:51:00,0.3
2010-01-01 02:09:00,0.3
2010-01-01 02:51:00,0.5


In [18]:
wind.head()

,Spd[Wind]
datetime,
2010-01-01 01:00:00,1.5
2010-01-01 01:15:00,0.0
2010-01-01 01:51:00,0.0
2010-01-01 02:09:00,0.0
2010-01-01 02:51:00,0.0
